In [2]:
# -*- coding: utf-8 -*-
#加载所需的库

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

import tensorflow
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [13]:
#加载数据
#path = "C:/Users/Administrator/Desktop/Python Process/AA.xlsx"
path = "C:/Users/Administrator/Desktop/Python Process/CC.xlsx"
file = pd.read_excel(path,sheet_name='Sheet2')
data = file.copy()
#重置行索引
data.set_index('年份',inplace=True)
data = round(data,1)
data.head(5)

,国内生产总值,第三产业增加值,国内旅游人数,国际旅游外汇收入,旅游周转里程,社会消费品销售额,住宿营业额,餐饮营业额
年份,,,,,,,,
1981,4935.8,1121.1,300.0,785.0,2500.0,2350.0,500.0,200.0
1982,5373.4,1214.0,315.0,843.0,2743.0,2570.0,518.0,210.0
1983,6020.9,1397.1,335.0,941.0,3095.0,2849.4,545.0,225.0
1984,7278.5,1858.2,351.7,1131.0,3620.0,3376.4,566.0,236.7
1985,9098.9,2670.8,369.2,1250.0,4435.4,4305.0,588.5,249.2


In [15]:
#数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaler = scaler.fit_transform(data)
data_scaler = pd.DataFrame(data_scaler,columns=data.columns)
data_scaler = round(data_scaler,3)
data_scaler.to_excel('C:/Users/Administrator/Desktop/Python Process/data_scaler1.xlsx')
data_scaler.head()

,国内生产总值,第三产业增加值,国内旅游人数,国际旅游外汇收入,旅游周转里程,社会消费品销售额,住宿营业额,餐饮营业额
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.000,0.003,0.000,0.007,0.001,0.005,0.002
2,0.001,0.001,0.006,0.001,0.018,0.001,0.012,0.004
3,0.002,0.001,0.009,0.003,0.034,0.003,0.017,0.006
4,0.004,0.003,0.012,0.004,0.059,0.005,0.023,0.008


In [20]:
col = ['GDP','AVTI','DTA','FEII','TMTT','SCGS','LIT','CIT']
data_ct = data_scaler.copy()
data_ct.columns = col
data_ct.head(5)

,GDP,AVTI,DTA,FEII,TMTT,SCGS,LIT,CIT
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.000,0.000,0.003,0.000,0.007,0.001,0.005,0.002
2,0.001,0.001,0.006,0.001,0.018,0.001,0.012,0.004
3,0.002,0.001,0.009,0.003,0.034,0.003,0.017,0.006
4,0.004,0.003,0.012,0.004,0.059,0.005,0.023,0.008


In [42]:
from statsmodels.tsa.stattools import adfuller
GDP_diff  = np.diff(data_ct.iloc[:,0],2)
AVTI_diff = np.diff(data_ct.iloc[:,1],2)
DTA_diff  = np.diff(data_ct.iloc[:,2],2)
FEII_diff = np.diff(data_ct.iloc[:,3],2)
TMTT_diff = np.diff(data_ct.iloc[:,4],2)
SCGS_diff = np.diff(data_ct.iloc[:,5],2)
LIT_diff  = np.diff(data_ct.iloc[:,6],2)
CIT_diff  = np.diff(data_ct.iloc[:,7],2)

In [43]:
print(adfuller(GDP_diff))
print(adfuller(AVTI_diff))
print(adfuller(DTA_diff))
print(adfuller(FEII_diff))
print(adfuller(TMTT_diff))
print(adfuller(SCGS_diff))
print(adfuller(LIT_diff))
print(adfuller(CIT_diff))

(-7.684612479395797, 1.4729117081157073e-11, 1, 35, {'1%': -3.6327426647230316, '5%': -2.9485102040816327, '10%': -2.6130173469387756}, -157.72473547874836)
(-5.350507890880732, 4.29972541902525e-06, 1, 35, {'1%': -3.6327426647230316, '5%': -2.9485102040816327, '10%': -2.6130173469387756}, -179.9154900694615)
(-1.1366018811108263, 0.7002570521464692, 6, 30, {'1%': -3.6699197407407405, '5%': -2.9640707407407407, '10%': -2.621171111111111}, -137.23845798587627)
(1.3534214282234338, 0.9968919201294241, 9, 27, {'1%': -3.6996079738860943, '5%': -2.9764303469999494, '10%': -2.627601001371742}, -92.86730968908807)
(-3.994726457682034, 0.0014383782008656116, 8, 28, {'1%': -3.6889256286443146, '5%': -2.9719894897959187, '10%': -2.6252957653061224}, -81.15943137526783)
(-5.9250808257449865, 2.454540335482682e-07, 0, 36, {'1%': -3.626651907578875, '5%': -2.9459512825788754, '10%': -2.6116707716049383}, -194.64950446797474)
(-1.5802100669302508, 0.4935265540816056, 10, 26, {'1%': -3.71121230086481

In [44]:
from statsmodels.tsa.stattools import coint
"""
maxlag (无或int)

adfuller的关键字，最大或给定滞后数

autolag (string)

adfuller的关键字，滞后选择标准。

如果为None，则使用maxlag滞后而不进行滞后搜索

如果是’AIC’(默认值)或’BIC’，则选择滞后数以最小化相应的信息标准
"""
print(coint(GDP_diff, AVTI_diff))

(-0.4018957264824984, 0.9706266213745081, array([-4.22652904, -3.51111966, -3.16435988]))
